In [ ]:
import pandas as pd
import numpy as np

import os.path as op
import sys

sys.path.append(op.abspath('../code'))

import plotting_tools

import importlib

import data_loaders as dload
import clusterize as cluster

In [ ]:
path_to_Piriform_neurons = '/mnt/c/Users/Cionkito/Documents/Brainhack2022/Piriform_neurons'
path_to_xls = op.join(path_to_Piriform_neurons, 'data')
my_matrix = dload.get_neuron_matrix(path_to_xls)
my_matrix

In [ ]:
is_na_idx = np.where(my_matrix.isna())[0]
my_matrix.iloc[is_na_idx]

In [ ]:
cluster_df = my_matrix.dropna(axis = 0).apply(cluster.normalize).dropna(axis = 1)

feat_to_del = ['Filament BoundingBoxAA Length X', 'Filament BoundingBoxAA Length Y', 'Filament BoundingBoxAA Length Z']
cluster_df = cluster_df.drop(columns = feat_to_del)

print(cluster_df.shape)

cluster_df

In [ ]:
importlib.reload(cluster)

show_figures = True
save_figures = False

cluster_model, label_2_color = cluster.hierarchical_cstm(cluster_df, d_thresh = 11.9, figsize = (30, 10), plot = show_figures)

labels = cluster_model.labels_
n_clusters = labels.max() + 1

if save_figures:
    plotting_tools.save_figure(op.join(path_to_Piriform_neurons, f'figures/hierarchical-tree-{n_clusters:02d}clusters.png'))

centroids = cluster.get_centroids(cluster_model, cluster_df, col = label_2_color, plot = show_figures)

if save_figures:
    plotting_tools.save_figure(op.join(path_to_Piriform_neurons, f'figures/hierarchical-centroids-{n_clusters:02d}clusters.png'))

dist_to_centroid = cluster.get_centroid_distance(cluster_model, cluster_df, centroids)

In [ ]:
print('List of neurons in each cluster:')

for i in range(n_clusters):
    print(f'cluster #{i + 1} :', cluster_df.loc[labels == i].index.tolist())

label_save_df = cluster_df.iloc[:, :1].copy()
label_save_df[f'labels-{n_clusters}k'] = labels

label_save_df = label_save_df.drop(columns = ['Filament BoundingBoxOO Length A'])

# Un-comment to save cluster labels (from 0 to n_cluster-1) in .csv file
# where each row is a neuron with an ID and its corresponding label

#label_save_df.to_csv('/Users/AlexCionca/Documents/BrainHack-2023/Piriform_neurons/neuron_clustering.csv')

In [ ]:
print('Features with the least in-cluster variability:')

for i, cluster_features in enumerate(dist_to_centroid.feat_by_radius):
    print(f'cluster #{i + 1:d}', cluster_features[:4])